<a href="https://colab.research.google.com/github/Riipou/First-order/blob/main/python/Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [323]:
import numpy as np
from scipy.io import loadmat
import numpy as np
import csv

# Chargement et pré-traitement des données

In [324]:
def one_hot_encoding(y):
    new_y = []
    for label in y:
        label_vector = np.zeros(20)
        label_vector[label - 1] = 1
        new_y.append(label_vector)
    y = np.array(new_y)
    return y


def shuffle(X, y):
    index = np.arange(np.shape(X)[1])
    np.random.shuffle(index)
    X = X[:, index]
    y = y.T[:, index]
    return X, y


def split_data(X, y, split_value=0.8):
    m = X.shape[1]
    m = int(m * split_value)
    X_train = X[:, :m]
    y_train = y[:m, :]
    X_test = X[:, m:]
    y_test = y[m:, :]
    return X_train, y_train, X_test, y_test


In [325]:
# Charger le fichier .mat
nom_fichier_mat = 'data_doc.mat'
donnees = loadmat(nom_fichier_mat)

# Xts combien de fois le mot dans le texte
if 'Xts' in donnees:
    Xts = donnees['Xts']
else:
    print("La variable 'Xts' n'a pas été trouvée dans le fichier.")

# id texte avec classe
if 'yts' in donnees:
    yts = donnees['yts']
else:
    print("La variable 'yts' n'a pas été trouvée dans le fichier.")

# Xvr combien de fois le mot dans le texte
if 'Xvr' in donnees:
    Xvr = donnees['Xvr']
else:
    print("La variable 'Xvr' n'a pas été trouvée dans le fichier.")

yts = one_hot_encoding(yts)
Xts, yts = shuffle(Xts, yts)
# X_train, y_train, X_test, y_test = split_data(Xts, yts)

#Descente du gradient

In [326]:
def loss_function(y, y_pred):
    n = y.shape[1]
    loss = (np.linalg.norm(y - y_pred, 'fro') ** 2) / n
    return loss

def sigmoid(X, a):
    return 1 / (1 + np.exp(-a * X))

def sigmoid_derivate(X, a):
    return a * sigmoid(X, a) * (1 - sigmoid(X, a))

def relu(x, a):
    return np.maximum(0, a * x)

def initialization_random(m, p):
    W = np.random.rand(m, p) - 0.5
    b = np.random.rand(p) - 0.5
    return W, b

def initialization_xavier(m, p):
    W = np.random.randn(m, p) * np.sqrt(2 / (m + p))
    b = np.zeros((p,))
    return W, b

def initialization_lecun(m, p):
    W = np.random.randn(m, p) * np.sqrt(1 / m)
    b = np.zeros((p,))
    return W, b

def grad_W(W, B, a, X, y, n):
    return X @ ((1 / n) * np.multiply(sigmoid(W.T @ X + B, a) - y, sigmoid_derivate(W.T @ X + B, a))).T

def grad_B(W, B, a, X, y, n):
    v1 = np.ones((n, 1))
    return ((1 / n) * np.multiply(sigmoid(W.T @ X + B, a) - y, sigmoid_derivate(W.T @ X + B, a))) @ v1

def evol_beta_paul(k):
    return  (k-1)/(k+2)

def evol_beta_nesterov(alpha_b):
    alpha_b2 = (np.sqrt((alpha_b**4) + 4*alpha_b**2) - alpha_b**2) / 2
    beta = (alpha_b * (1 - alpha_b)) / (alpha_b**2 + alpha_b2)
    return beta, alpha_b2

# Descente de gradient

In [327]:
def update_W(alpha, W, B, a, X, y):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5
    W2 = W - alpha * grad_W(W, B, a, X, y, n)
    while loss_function(y, sigmoid(W2.T @ X + B, a)) > loss and alpha != (alpha/2):
        alpha /= 2
        W2 = W - alpha * grad_W(W, B, a, X, y, n)
    return W2, alpha


def update_B(alpha, W, B, a, X, y):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5
    b = B[:, 1]
    b = b.reshape(-1, 1)
    b2 = b - alpha * grad_B(W, B, a, X, y, n)
    B2 = np.tile(b2.reshape(-1), (n, 1)).T
    while loss_function(y, sigmoid(W.T @ X + B2, a)) > loss and alpha != (alpha/2):
        alpha /= 2
        b2 = b - alpha * grad_B(W, B, a, X, y, n)
        B2 = np.tile(b2.reshape(-1), (n, 1)).T
    return B2, alpha

def update_W_Polyak (alpha, W, B, a, X, y, W_prev, alpha_b, beta):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5

    # Mise à jour des poids en utilisant la méthode de Polyak's
    W2 = W - alpha * grad_W(W, B, a, X, y, n) + beta * (W - W_prev)

    if loss_function(y, sigmoid(W2.T @ X + B, a)) > loss :
        alpha /= 2

        W2 = W - alpha * grad_W(W, B, a, X, y, n)
        print("testW")

    return W2, alpha, W, alpha_b


def update_B_Polyak (alpha, W, B, a, X, y, B_prev, alpha_b, beta):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5

    b = B[:, 1]
    b = b.reshape(-1, 1)
    b_prev = B_prev[:, 1]
    b_prev = b_prev.reshape(-1,1)

    # Mise à jour des poids en utilisant la méthode de Polyak's
    b2 = b - alpha * grad_B(W, B, a, X, y, n) + beta * (b - b_prev)
    B2 = np.tile(b2.reshape(-1), (n, 1)).T

    if loss_function(y, sigmoid(W.T @ X + B2, a)) > loss:
        alpha /= 2

        b2 = b - alpha * grad_B(W, B, a, X, y, n)
        B2 = np.tile(b2.reshape(-1), (n, 1)).T

    return B2, alpha, B, alpha_b

def update_W_Nesterov(alpha, W, B, a, X, y, W_prev, alpha_b, beta):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5

    # Mise à jour des poids en utilisant la méthode de Nesterov
    shifted_point = W - beta * (W - W_prev)
    W2 = W - alpha * grad_W(shifted_point, B, a, X, y, n) + beta * (W - W_prev)

    if loss_function(y, sigmoid(W2.T @ X + B, a)) > loss :
      alpha /= 2
      W2 = W - alpha * grad_W(W, B, a, X, y, n)

    return W2, alpha, W, alpha_b


def update_B_Nesterov(alpha, W, B, a, X, y, B_prev, alpha_b, beta):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5

    b = B[:, 1]
    b = b.reshape(-1, 1)
    b_prev = B_prev[:, 1]
    b_prev = b_prev.reshape(-1,1)

    # Mise à jour des poids en utilisant la méthode de Nesterov
    shifted_point = B - beta * (B - B_prev)
    b2 = b - alpha * grad_B(W, shifted_point, a, X, y, n) + beta * (b - b_prev)
    B2 = np.tile(b2.reshape(-1), (n, 1)).T

    if loss_function(y, sigmoid(W.T @ X + B2, a)) > loss:
      alpha /= 2
      b2 = b - alpha * grad_B(W, B, a, X, y, n)
      B2 = np.tile(b2.reshape(-1), (n, 1)).T

    return B2, alpha, B, alpha_b



def linear_model(m, n, p, max_iterations, a, stop_condition, X, y, init, accel = "none", evol_b = "paul"):

    if init == "random":
        # Initialisation random pour les poids (76,987)
        W, b = initialization_random(m, p)
    elif init == "xavier":
        # Initialisation de Xavier/Glorot pour les poids (73,513)
        W, b = initialization_xavier(m, p)
    elif init == "lecun":
        # Initialisation de Lecun pour les poids (77,388)
        W, b = initialization_lecun(m, p)

    B = np.tile(b, (n, 1)).T
    alpha_W = 0.1 * np.linalg.norm(W, 'fro') / np.linalg.norm(grad_W(W, B, a, X, y, n), 'fro')
    alpha_B = 0.1 * np.linalg.norm(B, 'fro') / np.linalg.norm(grad_B(W, B, a, X, y, n), 'fro')

    if accel == "none" :
      for _ in range(max_iterations):
          loss_prev = loss_function(y, sigmoid(W.T @ X + B, 1))
          W, alpha_W = update_W(alpha_W, W, B, a, X, y)
          B, alpha_B = update_B(alpha_B, W, B, a, X, y)
          loss = loss_function(y, sigmoid(W.T @ X + B, 1))
          print(loss)
          if abs(loss - loss_prev) < stop_condition:
              print("break")
              break
      return W, b

    elif accel == "polyak" :
      W_prev = W
      B_prev = B
      alpha_b = 0.5
      for k in range(1,max_iterations+1):

          if evol_b == "const" :
              beta = 0.5

          if evol_b == "paul" :
              beta = evol_beta_paul(k)

          if evol_b == "nesterov" :
              beta, alpha_b = evol_beta_nesterov(alpha_b)

          loss_prev = loss_function(y, sigmoid(W.T @ X + B, 1))
          W, alpha_W, W_prev, alpha_b = update_W_Polyak(alpha_W, W, B, a, X, y, W_prev, alpha_b, beta)
          B, alpha_B, B_prev, alpha_b = update_B_Polyak(alpha_B, W, B, a, X, y, B_prev, alpha_b, beta)
          loss = loss_function(y, sigmoid(W.T @ X + B, 1))
          print(loss)
          if abs(loss - loss_prev) < stop_condition:
              print("break")
              break
      return W, b


    elif accel == "nesterov" :
      W_prev = W
      B_prev = B
      alpha_b = 0.5
      for k in range(1,max_iterations+1):

          if evol_b == "const" :
              beta = 0.5

          if evol_b == "paul" :
              beta = evol_beta_paul(k)

          if evol_b == "nesterov" :
              beta, alpha_b = evol_beta_nesterov(alpha_b)

          loss_prev = loss_function(y, sigmoid(W.T @ X + B, 1))
          W, alpha_W, W_prev, alpha_b = update_W_Nesterov(alpha_W, W, B, a, X, y, W_prev, alpha_b, beta)
          B, alpha_B, B_prev, alpha_b = update_B_Nesterov(alpha_B, W, B, a, X, y, B_prev, alpha_b, beta)
          loss = loss_function(y, sigmoid(W.T @ X + B, 1))
          print(loss)
          if abs(loss - loss_prev) < stop_condition:
              print("break")
              break
      return W, b



In [334]:
m, n = Xts.shape
p, _ = yts.shape

# Définition des variables
num_iterations = 10000
a = 0.05
stop_condition = 1e-5
# Choisir initialisation : random, xavier, lecun
initialisation = "lecun"
# Choisir acceleration : none, polyak, nesterov
acceleration = "nesterov"
# Choisir evolution beta : const, paul , nesterov
evol_b = "paul"

W, b = linear_model(m, n, p, num_iterations, a, stop_condition, Xts, yts, initialisation, acceleration, evol_b)



3.8173265410244266
2.702962703753734
1.9050610735588387
1.4596191254725956
1.2373969280037347
1.123187822840384
1.0624592662300913
1.0306951303223602


<ipython-input-326-2c9294a05fed>:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-a * X))


1.013719125095668
1.0037942032582015
0.9968230407099992
0.9902469055717864
0.9819843171583211
0.9703333926586235
0.9512146652978524
0.9233864186414592
0.8837412428706395
0.8324326796423445
0.7684504353883923


KeyboardInterrupt: 

# Evaluation des données

# Création validation

In [ ]:
def reverse_one_hot_encoding(y):
    y = np.argmax(y, axis=0)+1
    return y

def test(X, W, b):
    _, n = X.shape
    B = np.tile(b, (n, 1)).T
    y = sigmoid(W.T @ X + B, 1)
    y = reverse_one_hot_encoding(y)
    ids = np.arange(1, len(y)+1)
    data = np.column_stack((ids, 1+100 * y))
    nom_fichier = "test2.csv"

    header = ["id","class"]
    with open(nom_fichier, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        writer.writerows(data)
    return 0

In [ ]:
test(Xvr, W, b)